# Séance 3: TP1 - Pipeline de Classification Binaire

::: {.callout-note icon=false}
## Informations de la séance
- **Type**: Travaux Pratiques
- **Durée**: 2h
- **Objectifs**: Obj6, Obj7
- **Dataset**: Titanic (prédiction de survie)
:::

## Objectifs du TP

À la fin de ce TP, vous serez capable de:

1. Charger et explorer un dataset
2. Préparer les données pour l'apprentissage
3. Créer un pipeline de prétraitement avec Scikit-learn
4. Entraîner un modèle de classification binaire
5. Évaluer les performances du modèle

## 1. Configuration de l'Environnement

In [ ]:
# Installation des bibliothèques (si nécessaire)
# !pip install scikit-learn pandas numpy matplotlib seaborn

# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline

# Configuration
plt.style.use('default')
sns.set_palette("husl")
np.random.seed(42)

print("✓ Bibliothèques importées avec succès")

## 2. Chargement et Exploration des Données

### 2.1 Chargement du Dataset Titanic

In [ ]:
# Chargement depuis seaborn
titanic = sns.load_dataset('titanic')

# Affichage des premières lignes
print("Aperçu des données:")
print(titanic.head())

print(f"\nDimensions: {titanic.shape}")
print(f"Colonnes: {titanic.columns.tolist()}")

### 2.2 Exploration Initiale

In [ ]:
# Informations générales
print("Informations sur le dataset:")
print(titanic.info())

print("\nStatistiques descriptives:")
print(titanic.describe())

# Vérification des valeurs manquantes
print("\nValeurs manquantes:")
print(titanic.isnull().sum())

# Distribution de la variable cible
print("\nDistribution de la survie:")
print(titanic['survived'].value_counts())
print(f"\nTaux de survie: {titanic['survived'].mean():.2%}")

### 2.3 Visualisations Exploratoires

In [ ]:
# Figure 1: Distribution de la survie
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Survie globale
axes[0, 0].pie(
    titanic['survived'].value_counts(), 
    labels=['Décédé', 'Survivant'],
    autopct='%1.1f%%',
    startangle=90,
    colors=['#ff6b6b', '#51cf66']
)
axes[0, 0].set_title('Distribution de la Survie')

# Survie par sexe
survival_by_sex = titanic.groupby(['sex', 'survived']).size().unstack()
survival_by_sex.plot(kind='bar', ax=axes[0, 1], color=['#ff6b6b', '#51cf66'])
axes[0, 1].set_title('Survie par Sexe')
axes[0, 1].set_xlabel('Sexe')
axes[0, 1].set_ylabel('Nombre de passagers')
axes[0, 1].legend(['Décédé', 'Survivant'])
axes[0, 1].tick_params(axis='x', rotation=0)

# Survie par classe
survival_by_class = titanic.groupby(['pclass', 'survived']).size().unstack()
survival_by_class.plot(kind='bar', ax=axes[1, 0], color=['#ff6b6b', '#51cf66'])
axes[1, 0].set_title('Survie par Classe')
axes[1, 0].set_xlabel('Classe')
axes[1, 0].set_ylabel('Nombre de passagers')
axes[1, 0].legend(['Décédé', 'Survivant'])

# Distribution de l'âge
axes[1, 1].hist(titanic[titanic['survived']==0]['age'].dropna(), 
                alpha=0.5, label='Décédé', bins=30, color='#ff6b6b')
axes[1, 1].hist(titanic[titanic['survived']==1]['age'].dropna(), 
                alpha=0.5, label='Survivant', bins=30, color='#51cf66')
axes[1, 1].set_title("Distribution de l'âge par survie")
axes[1, 1].set_xlabel('Âge')
axes[1, 1].set_ylabel('Fréquence')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

## 3. Préparation des Données

### 3.1 Sélection des Features

In [ ]:
features = ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']
target = 'survived'

df = titanic[features + [target]].copy()

# Valeurs manquantes
df['age']      = df['age'].fillna(df['age'].median())
df['embarked'] = df['embarked'].fillna(df['embarked'].mode()[0])
df['fare']     = df['fare'].fillna(df['fare'].median())

# Encodage sex → numérique (OBLIGATOIRE avant pipeline)
df['sex'] = df['sex'].map({'male': 0, 'female': 1})

# One-Hot encoding embarked
df = pd.get_dummies(df, columns=['embarked'], prefix='embarked', drop_first=True)

# Séparation X / y
X = df.drop('survived', axis=1)
y = df['survived']

# Vérification : aucune colonne string ne doit rester
print(X.dtypes)
print(f"\nShape: {X.shape}")
print(f"Types problématiques: {X.select_dtypes(include='object').columns.tolist()}")

### 3.2 Traitement des Valeurs Manquantes

In [ ]:
# Stratégies de traitement
# 1. Age: remplir avec la médiane
df['age'].fillna(df['age'].median(), inplace=True)

# 2. Embarked: remplir avec le mode (valeur la plus fréquente)
df['embarked'].fillna(df['embarked'].mode()[0], inplace=True)

# 3. Fare: remplir avec la médiane (si manquant)
df['fare'].fillna(df['fare'].median(), inplace=True)

# Vérification
print("Après traitement:")
print(df.isnull().sum())

### 3.3 Encodage des Variables Catégorielles

In [ ]:
# Encodage de 'sex'
df['sex'] = df['sex'].map({'male': 0, 'female': 1})

# Encodage de 'embarked' (One-Hot Encoding)
df = pd.get_dummies(df, columns=['embarked'], prefix='embarked', drop_first=True)

print("Dataset après encodage:")
print(df.head())
print(f"\nNouvelles dimensions: {df.shape}")

### 3.4 Séparation Features / Target

In [ ]:
# Séparation X (features) et y (target)
X = df.drop('survived', axis=1)
y = df['survived']

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"\nFeatures utilisées:\n{X.columns.tolist()}")

## 4. Split Train/Validation/Test

### 4.1 Split Train/Test

In [ ]:
# Split 80/20
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,      # 20% pour le test
    random_state=42,    # reproductibilité
    stratify=y          # préserver la distribution des classes
)

print(f"Train set: {X_train.shape}")
print(f"Test set:  {X_test.shape}")

# Vérification de la distribution
print(f"\nDistribution train: {y_train.value_counts(normalize=True)}")
print(f"Distribution test:  {y_test.value_counts(normalize=True)}")

### 4.2 Split Train/Validation (optionnel)

In [ ]:
# Optionnel: créer un ensemble de validation
X_train_full, X_val, y_train_full, y_val = train_test_split(
    X_train, y_train,
    test_size=0.2,  # 20% du train pour validation
    random_state=42,
    stratify=y_train
)

print(f"Train full: {X_train_full.shape}")
print(f"Validation: {X_val.shape}")
print(f"Test:       {X_test.shape}")

## 5. Pipeline de Prétraitement et Entraînement

### 5.1 Création du Pipeline

In [ ]:
# Pipeline: Standardisation + Modèle
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Étape 1: Standardisation
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))  # Étape 2: Modèle
])

print("Pipeline créé:")
print(pipeline)

### 5.2 Entraînement du Modèle

In [ ]:
# Entraînement
print("Entraînement en cours...")
pipeline.fit(X_train, y_train)
print("✓ Entraînement terminé")

# Prédictions
y_train_pred = pipeline.predict(X_train)
y_test_pred = pipeline.predict(X_test)

print("✓ Prédictions effectuées")

## 6. Évaluation Initiale

### 6.1 Accuracy

In [ ]:
# Calcul de l'accuracy
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Accuracy Train: {train_accuracy:.4f} ({train_accuracy*100:.2f}%)")
print(f"Accuracy Test:  {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

# Analyse de l'overfitting
diff = train_accuracy - test_accuracy
print(f"\nDifférence Train-Test: {diff:.4f}")
if diff < 0.05:
    print("→ Bon équilibre biais-variance")
elif diff < 0.10:
    print("→ Léger overfitting")
else:
    print("→ Overfitting significatif")

### 6.2 Matrice de Confusion

In [ ]:
# Calcul de la matrice de confusion
cm = confusion_matrix(y_test, y_test_pred)

# Visualisation
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Décédé', 'Survivant'],
            yticklabels=['Décédé', 'Survivant'])
plt.title('Matrice de Confusion - Test Set')
plt.ylabel('Vraie Classe')
plt.xlabel('Classe Prédite')
plt.tight_layout()
plt.show()

# Interprétation
tn, fp, fn, tp = cm.ravel()
print(f"\nVrais Négatifs (TN):  {tn}")
print(f"Faux Positifs (FP):   {fp}")
print(f"Faux Négatifs (FN):   {fn}")
print(f"Vrais Positifs (TP):  {tp}")

### 6.3 Rapport de Classification

In [ ]:
# Rapport détaillé
print("\nRapport de Classification:")
print(classification_report(y_test, y_test_pred, 
                          target_names=['Décédé', 'Survivant']))

## 7. Comparaison de Plusieurs Modèles

In [ ]:
# Définition des modèles
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Decision Tree': DecisionTreeClassifier(max_depth=5, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
}

# Entraînement et évaluation
results = {}
for name, model in models.items():
    # Pipeline pour chaque modèle
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', model)
    ])
    
    # Entraînement
    pipe.fit(X_train, y_train)
    
    # Évaluation
    train_score = pipe.score(X_train, y_train)
    test_score = pipe.score(X_test, y_test)
    
    results[name] = {
        'train': train_score,
        'test': test_score,
        'diff': train_score - test_score
    }
    
    print(f"\n{name}:")
    print(f"  Train Accuracy: {train_score:.4f}")
    print(f"  Test Accuracy:  {test_score:.4f}")
    print(f"  Différence:     {train_score - test_score:.4f}")

# Visualisation comparative
df_results = pd.DataFrame(results).T
df_results[['train', 'test']].plot(kind='bar', figsize=(10, 6))
plt.title('Comparaison des Performances des Modèles')
plt.xlabel('Modèle')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Test'])
plt.xticks(rotation=45, ha='right')
plt.ylim([0, 1])
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 8. Analyse des Prédictions

### 8.1 Exemples de Prédictions

In [ ]:
# Prédictions avec probabilités
y_proba = pipeline.predict_proba(X_test)

# Affichage de quelques exemples
n_samples = 5
indices = np.random.choice(len(X_test), n_samples, replace=False)

print("Exemples de prédictions:\n")
for idx in indices:
    actual = y_test.iloc[idx]
    predicted = y_test_pred[idx]
    proba = y_proba[idx]
    
    print(f"Passager {idx}:")
    print(f"  Vraie classe:     {'Survivant' if actual == 1 else 'Décédé'}")
    print(f"  Prédiction:       {'Survivant' if predicted == 1 else 'Décédé'}")
    print(f"  Probabilités:     Décédé={proba[0]:.2%}, Survivant={proba[1]:.2%}")
    print(f"  Correct:          {'+' if actual == predicted else '+'}")
    print()

### 8.2 Analyse des Erreurs

In [ ]:
# Identification des erreurs
errors = X_test[y_test != y_test_pred].copy()
errors['actual'] = y_test[y_test != y_test_pred]
errors['predicted'] = y_test_pred[y_test != y_test_pred]

print(f"Nombre d'erreurs: {len(errors)}")
print(f"Taux d'erreur: {len(errors)/len(X_test):.2%}")

print("\nQuelques erreurs:")
print(errors.head())

# Analyse des caractéristiques des erreurs
print("\nCaractéristiques moyennes des erreurs vs correctes:")
correct = X_test[y_test == y_test_pred]

comparison = pd.DataFrame({
    'Erreurs': errors.drop(['actual', 'predicted'], axis=1).mean(),
    'Correctes': correct.mean()
})
print(comparison)

## Exercices Pratiques

### Exercice 1: Feature Engineering

Créez une nouvelle feature `family_size` = `sibsp` + `parch` + 1, puis ré-entraînez le modèle. La performance s'améliore-t-elle ?

#### Solution

In [ ]:
# Création de la nouvelle feature
df['family_size'] = df['sibsp'] + df['parch'] + 1

# Refaire le split et l'entraînement
X_new = df.drop('survived', axis=1)
y_new = df['survived']

X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(
    X_new, y_new, test_size=0.2, random_state=42, stratify=y_new
)

pipeline_new = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))
])

pipeline_new.fit(X_train_new, y_train_new)
new_score = pipeline_new.score(X_test_new, y_test_new)

print(f"Accuracy avec family_size: {new_score:.4f}")
print(f"Accuracy sans family_size: {test_accuracy:.4f}")
print(f"Amélioration: {new_score - test_accuracy:.4f}")

### Exercice 2: Optimisation des Hyperparamètres

Testez différentes valeurs de `max_depth` pour le Decision Tree (3, 5, 7, 10, None). Quelle valeur donne les meilleures performances sur le test set ?

#### Solution

In [ ]:
depths = [3, 5, 7, 10, None]
results_depth = []

for depth in depths:
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', DecisionTreeClassifier(max_depth=depth, random_state=42))
    ])
    
    pipe.fit(X_train, y_train)
    train_score = pipe.score(X_train, y_train)
    test_score = pipe.score(X_test, y_test)
    
    results_depth.append({
        'max_depth': str(depth) if depth is not None else 'None (pas de limite)',
        'train': train_score,
        'test': test_score,
        'diff': train_score - test_score
    })
    
df_depth = pd.DataFrame(results_depth)
print("\nRésultats pour différentes profondeurs:")
print(df_depth)

# CORRECTION 1: Exclure None de la recherche du meilleur modèle
df_without_none = df_depth[df_depth['max_depth'] != 'None (pas de limite)'].copy()
best_depth_info = df_without_none.loc[df_without_none['test'].idxmax()]

print(f"\nMeilleur max_depth (sans None): {best_depth_info['max_depth']}")
print(f"Test Accuracy: {best_depth_info['test']:.4f}")
print(f"Train Accuracy: {best_depth_info['train']:.4f}")
print(f"Différence Train-Test: {best_depth_info['diff']:.4f}")

# CORRECTION 2: Visualisation pour mieux comprendre
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Graphique 1: Scores Train/Test
axes[0].plot(range(len(df_depth)), df_depth['train'], 'o-', label='Train', linewidth=2)
axes[0].plot(range(len(df_depth)), df_depth['test'], 's-', label='Test', linewidth=2)
axes[0].set_xlabel('Profondeur')
axes[0].set_ylabel('Accuracy')
axes[0].set_title('Performance selon la profondeur')
axes[0].set_xticks(range(len(df_depth)))
axes[0].set_xticklabels(df_depth['max_depth'], rotation=45)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Graphique 2: Différence Train-Test (overfitting)
axes[1].bar(range(len(df_depth)), df_depth['diff'], color='red', alpha=0.6)
axes[1].set_xlabel('Profondeur')
axes[1].set_ylabel('Différence Train-Test')
axes[1].set_title('Niveau d\'overfitting')
axes[1].set_xticks(range(len(df_depth)))
axes[1].set_xticklabels(df_depth['max_depth'], rotation=45)
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# CORRECTION 3: Analyse détaillée
print("\n=== ANALYSE DÉTAILLÉE ===")
print("\n1. max_depth = 3:")
print(f"   - Test: {df_depth.loc[0, 'test']:.4f}, Train: {df_depth.loc[0, 'train']:.4f}")
print("   - Modèle simple, peu d'overfitting mais peut être sous-optimal")

print("\n2. max_depth = 5:")
print(f"   - Test: {df_depth.loc[1, 'test']:.4f}, Train: {df_depth.loc[1, 'train']:.4f}")
print("   - Bon compromis entre biais et variance")

print("\n3. max_depth = 7 ou 10:")
print(f"   - Test: {df_depth.loc[2, 'test']:.4f} et {df_depth.loc[3, 'test']:.4f}")
print("   - Performance légèrement meilleure mais risque d'overfitting accru")

print("\n4. max_depth = None (pas de limite):")
print(f"   - Test: {df_depth.loc[4, 'test']:.4f}, Train: {df_depth.loc[4, 'train']:.4f}")
print(f"   - Différence: {df_depth.loc[4, 'diff']:.4f}")
print("   - FORT RISQUE D'OVERFITTING sur de nouvelles données")
print("   - Le modèle mémorise le bruit des données d'entraînement")

# CORRECTION 4: Choix recommandé avec justification
print("\n=== RECOMMANDATION ===")
print(f"Le meilleur choix est max_depth = {best_depth_info['max_depth']} car:")
print("1. Il maximise le score sur le test set parmi les valeurs raisonnables")
print("2. La différence Train-Test est acceptable (équilibre biais-variance)")
print("3. Le modèle reste interprétable (arbre de taille raisonnable)")

# Validation supplémentaire avec validation croisée
from sklearn.model_selection import cross_val_score

if best_depth_info['max_depth'] != 'None (pas de limite)':
    depth_value = int(best_depth_info['max_depth'])
    
    # Validation croisée pour plus de robustesse
    cv_scores = cross_val_score(
        DecisionTreeClassifier(max_depth=depth_value, random_state=42),
        X_train, y_train,
        cv=5,
        scoring='accuracy'
    )
    
    print(f"\nValidation croisée (5 folds) pour max_depth = {depth_value}:")
    print(f"  Scores: {cv_scores}")
    print(f"  Moyenne: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

### Exercice 3: Analyse d'Importance

Pour le Random Forest, affichez l'importance des features. Quelles sont les 3 features les plus importantes ?

#### Solution

In [ ]:
# Entraîner Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Importance des features
importances = pd.DataFrame({
    'feature': X_train.columns,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)

print("Importance des features:")
print(importances)

# Visualisation
plt.figure(figsize=(10, 6))
plt.barh(importances['feature'], importances['importance'])
plt.xlabel('Importance')
plt.title('Importance des Features - Random Forest')
plt.tight_layout()
plt.show()

print(f"\nTop 3 features:")
print(importances.head(3))

In [ ]:
# ===========================================
# MODÈLE AVEC LES 4 FEATURES LES PLUS IMPORTANTES
# ===========================================

print("=" * 60)
print("MODÈLE SIMPLIFIÉ - 4 FEATURES PRINCIPALES")
print("=" * 60)

# 1. Sélection des 4 features les plus importantes
top_features = ['sex', 'fare', 'age', 'pclass']
print(f"\nFeatures sélectionnées: {top_features}")

# 2. Création du nouveau dataset avec uniquement ces features
X_top = df[top_features].copy()
y_top = df['survived']

print(f"\nNouvelles dimensions:")
print(f"X shape: {X_top.shape}")
print(f"y shape: {y_top.shape}")

# 3. Vérification des données
print("\nStatistiques des features sélectionnées:")
print(X_top.describe())

print("\nValeurs manquantes:")
print(X_top.isnull().sum())

# 4. Split Train/Test avec les nouvelles features
X_train_top, X_test_top, y_train_top, y_test_top = train_test_split(
    X_top, y_top, 
    test_size=0.2,
    random_state=42,
    stratify=y_top
)

print(f"\nSplit des données:")
print(f"Train: {X_train_top.shape}")
print(f"Test:  {X_test_top.shape}")

# 5. Pipeline optimisé pour les 4 features
pipeline_top = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(
        n_estimators=100,
        max_depth=5,
        random_state=42,
        max_features='sqrt'  # Adaptation pour peu de features
    ))
])

print("\n" + "=" * 40)
print("ENTRAÎNEMENT DU MODÈLE SIMPLIFIÉ")
print("=" * 40)

# Entraînement
pipeline_top.fit(X_train_top, y_train_top)

# Prédictions
y_train_pred_top = pipeline_top.predict(X_train_top)
y_test_pred_top = pipeline_top.predict(X_test_top)

# 6. Évaluation du modèle simplifié
print("\n" + "=" * 40)
print("ÉVALUATION DU MODÈLE SIMPLIFIÉ")
print("=" * 40)

# Accuracy
train_acc_top = accuracy_score(y_train_top, y_train_pred_top)
test_acc_top = accuracy_score(y_test_top, y_test_pred_top)

print(f"\nPerformance:")
print(f"Train Accuracy: {train_acc_top:.4f} ({train_acc_top*100:.2f}%)")
print(f"Test Accuracy:  {test_acc_top:.4f} ({test_acc_top*100:.2f}%)")

# Comparaison avec le modèle complet (si disponible)
try:
    print(f"\nComparaison avec modèle complet ({len(X.columns)} features):")
    print(f"Test Accuracy complet:  {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
    print(f"Test Accuracy simplifié: {test_acc_top:.4f} ({test_acc_top*100:.2f}%)")
    
    diff = test_acc_top - test_accuracy
    if diff > 0:
        print(f"Amélioration: +{diff:.4f} ({diff*100:.2f}%)")
    else:
        print(f"Dégradation: {diff:.4f} ({diff*100:.2f}%)")
except:
    pass

# 7. Matrice de confusion
print("\n" + "=" * 40)
print("MATRICE DE CONFUSION")
print("=" * 40)

cm_top = confusion_matrix(y_test_top, y_test_pred_top)
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

# Matrice de confusion
sns.heatmap(cm_top, annot=True, fmt='d', cmap='Blues', ax=ax[0],
            xticklabels=['Décédé', 'Survivant'],
            yticklabels=['Décédé', 'Survivant'])
ax[0].set_title('Matrice de Confusion - Modèle Simplifié')
ax[0].set_ylabel('Vraie Classe')
ax[0].set_xlabel('Classe Prédite')

# Rapport de classification
print("\nRapport de Classification:")
print(classification_report(y_test_top, y_test_pred_top, 
                          target_names=['Décédé', 'Survivant']))

# 8. Importance des features dans le nouveau modèle
print("\n" + "=" * 40)
print("IMPORTANCE DES FEATURES DANS LE MODÈLE SIMPLIFIÉ")
print("=" * 40)

rf_top = pipeline_top.named_steps['classifier']
importances_top = pd.DataFrame({
    'feature': top_features,
    'importance': rf_top.feature_importances_
}).sort_values('importance', ascending=False)

print("\nImportance relative:")
for idx, row in importances_top.iterrows():
    print(f"{row['feature']}: {row['importance']:.4f} ({row['importance']*100:.1f}%)")

# Visualisation
ax[1].barh(importances_top['feature'], importances_top['importance'], color='darkgreen')
ax[1].set_xlabel('Importance')
ax[1].set_title('Importance des Features - Modèle Simplifié')
ax[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

# 9. Analyse des probabilités de prédiction
print("\n" + "=" * 40)
print("ANALYSE DES PROBABILITÉS")
print("=" * 40)

y_proba_top = pipeline_top.predict_proba(X_test_top)

# Exemples de prédictions avec probabilités
print("\nExemples de prédictions (5 passagers aléatoires):")
np.random.seed(42)
sample_indices = np.random.choice(len(X_test_top), 5, replace=False)

for i, idx in enumerate(sample_indices):
    actual = y_test_top.iloc[idx]
    predicted = y_test_pred_top[idx]
    proba_survive = y_proba_top[idx, 1]
    
    print(f"\nPassager {i+1}:")
    print(f"  Features: ", end="")
    for feat in top_features:
        print(f"{feat}={X_test_top.iloc[idx][feat]:.2f} ", end="")
    
    print(f"\n  Probabilité survie: {proba_survive:.2%}")
    print(f"  Prédiction: {'Survivant' if predicted == 1 else 'Décédé'}")
    print(f"  Réel: {'Survivant' if actual == 1 else 'Décédé'}")
    print(f"  Correct: {'✓' if actual == predicted else '✗'}")

# 10. Interprétation des règles de décision
print("\n" + "=" * 40)
print("INTERPRÉTATION DES RÈGLES DE DÉCISION")
print("=" * 40)

# Exemple d'arbre de décision (premier arbre de la forêt)
from sklearn.tree import plot_tree

fig, ax = plt.subplots(figsize=(12, 8))
tree_to_plot = rf_top.estimators_[0]  # Premier arbre

plot_tree(tree_to_plot, 
          feature_names=top_features,
          class_names=['Décédé', 'Survivant'],
          filled=True, 
          rounded=True,
          ax=ax,
          max_depth=3)  # Limiter la profondeur pour lisibilité
plt.title("Exemple d'arbre de décision (premier arbre - profondeur limitée)")
plt.tight_layout()
plt.show()

# Règles simples extraites
print("\nRègles de décision principales:")
print("1. Si sex=female → forte probabilité de survie")
print("2. Si sex=male ET fare > moyenne → chance modérée")
print("3. Si sex=male ET fare bas ET age élevé → faible chance")
print("4. Classe 1 améliore les chances, surtout pour les hommes")

# 11. Validation croisée pour robustesse
print("\n" + "=" * 40)
print("VALIDATION CROISÉE")
print("=" * 40)

from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(
    pipeline_top,
    X_top,
    y_top,
    cv=5,
    scoring='accuracy'
)

print(f"\nScores de validation croisée (5 folds):")
print(f"  Scores individuels: {cv_scores}")
print(f"  Moyenne: {cv_scores.mean():.4f}")
print(f"  Écart-type: {cv_scores.std():.4f}")
print(f"  Intervalle de confiance (95%): {cv_scores.mean():.4f} ± {cv_scores.std()*2:.4f}")

# 12. Avantages du modèle simplifié
print("\n" + "=" * 40)
print("AVANTAGES DU MODÈLE SIMPLIFIÉ")
print("=" * 40)

print("\n✓ 1. SIMPLICITÉ:")
print(f"   - Features: {len(top_features)} vs {len(X.columns)}")
print(f"   - Complexité réduite")

print("\n✓ 2. INTERPRÉTABILITÉ:")
print(f"   - Compréhension facile des décisions")
print(f"   - Règles explicables aux non-experts")

print("\n✓ 3. ROBUSTESSE:")
print(f"   - Moins sensible au bruit")
print(f"   - Généralisation potentiellement meilleure")

print("\n✓ 4. EFFICACITÉ:")
print(f"   - Entraînement plus rapide")
print(f"   - Moins de mémoire requise")
print(f"   - Prédictions plus rapides")

# 13. Exemple d'utilisation pour de nouvelles prédictions
print("\n" + "=" * 40)
print("EXEMPLE D'UTILISATION PRATIQUE")
print("=" * 40)

# Fonction de prédiction simplifiée
def predict_survival(sex, fare, age, pclass):
    """
    Prédit la survie d'un passager
    
    Paramètres:
    - sex: 0 pour male, 1 pour female
    - fare: prix du billet
    - age: âge en années
    - pclass: classe (1, 2, 3)
    """
    # Création du DataFrame pour la prédiction
    new_passenger = pd.DataFrame({
        'sex': [sex],
        'fare': [fare],
        'age': [age],
        'pclass': [pclass]
    })
    
    # Prédiction
    survival_prob = pipeline_top.predict_proba(new_passenger)[0, 1]
    prediction = pipeline_top.predict(new_passenger)[0]
    
    return prediction, survival_prob

# Exemples pratiques
print("\nExemples de prédictions pour de nouveaux passagers:")

exemples = [
    {"sex": 1, "fare": 100, "age": 25, "pclass": 1, "desc": "Femme jeune, 1ère classe"},
    {"sex": 0, "fare": 10, "age": 45, "pclass": 3, "desc": "Homme adulte, 3ème classe"},
    {"sex": 1, "fare": 30, "age": 8, "pclass": 2, "desc": "Fille enfant, 2ème classe"},
    {"sex": 0, "fare": 80, "age": 60, "pclass": 1, "desc": "Homme âgé, 1ère classe"}
]

for exemple in exemples:
    pred, prob = predict_survival(
        exemple["sex"], 
        exemple["fare"], 
        exemple["age"], 
        exemple["pclass"]
    )
    
    sex_str = "Femme" if exemple["sex"] == 1 else "Homme"
    surv_str = "SURVIVANT" if pred == 1 else "DÉCÉDÉ"
    
    print(f"\n{exemple['desc']}:")
    print(f"  → Prédiction: {surv_str}")
    print(f"  → Probabilité de survie: {prob:.1%}")
    print(f"  → Facteurs favorables: ", end="")
    
    factors = []
    if exemple["sex"] == 1:
        factors.append("sexe féminin")
    if exemple["fare"] > 50:
        factors.append("billet cher")
    if exemple["age"] < 18:
        factors.append("enfant")
    if exemple["pclass"] == 1:
        factors.append("1ère classe")
    
    if factors:
        print(", ".join(factors))
    else:
        print("aucun (risque élevé)")

print("\n" + "=" * 60)
print("CONCLUSION : MODÈLE SIMPLIFIÉ VALIDÉ")
print("=" * 60)

print(f"""
Résumé final:
• Features utilisées: {len(top_features)} (sur {len(X.columns)} initiales)
• Test Accuracy: {test_acc_top:.2%}
• Complexité: TRÈS RÉDUITE
• Interprétabilité: EXCELLENTE

Recommandation:
✓ Ce modèle simplifié est suffisant pour la majorité des cas d'usage
✓ Il est plus facile à déployer et maintenir
✓ Les décisions sont compréhensibles par les humains
✓ Performance similaire au modèle complexe
""")

In [ ]:
# ===========================================
# VERSION ULTRA-SIMPLE : LOGISTIC REGRESSION
# ===========================================

print("\n" + "=" * 60)
print("VERSION ULTRA-SIMPLE : LOGISTIC REGRESSION")
print("=" * 60)

# Pipeline ultra-simple
pipeline_simple = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))
])

# Entraînement
pipeline_simple.fit(X_train_top, y_train_top)

# Évaluation
y_pred_simple = pipeline_simple.predict(X_test_top)
acc_simple = accuracy_score(y_test_top, y_pred_simple)

print(f"\nAccuracy avec Logistic Regression: {acc_simple:.4f} ({acc_simple*100:.2f}%)")

# Coefficients pour interprétation
lr_model = pipeline_simple.named_steps['classifier']
coefficients = pd.DataFrame({
    'feature': top_features,
    'coefficient': lr_model.coef_[0],
    'impact': np.exp(lr_model.coef_[0])  # Odds ratio
}).sort_values('coefficient', ascending=False)

print("\nCoefficients (interprétation):")
print(coefficients)

print("\nInterprétation:")
print("• Coefficient POSITIF = augmente la probabilité de survie")
print("• Coefficient NÉGATIF = diminue la probabilité de survie")
print("• Odds Ratio > 1 = multiplie les chances de survie")
print("• Odds Ratio < 1 = divise les chances de survie")

# Exemple : impact d'une augmentation d'une unité
print("\nImpact pratique:")
for _, row in coefficients.iterrows():
    odds = row['impact']
    if odds > 1:
        change = f"augmente de {(odds-1)*100:.0f}%"
    else:
        change = f"diminue de {(1-odds)*100:.0f}%"
    
    print(f"  • {row['feature']}: 1 unité supplémentaire {change} les chances de survie")

## Résumé du TP

### Ce que vous avez appris

1. **Chargement et exploration** de données avec pandas
2. **Prétraitement** des données:
   - Traitement des valeurs manquantes
   - Encodage des variables catégorielles
   - Standardisation
3. **Pipeline Scikit-learn** pour automatiser le workflow
4. **Split Train/Test** avec stratification
5. **Entraînement et évaluation** de modèles de classification
6. **Comparaison** de plusieurs algorithmes
7. **Analyse des résultats** et des erreurs

## Checklist de Validation

- [ ] Dataset chargé et exploré
- [ ] Valeurs manquantes traitées
- [ ] Variables catégorielles encodées
- [ ] Pipeline créé avec StandardScaler
- [ ] Modèle entraîné avec succès
- [ ] Accuracy calculée (train et test)
- [ ] Matrice de confusion générée
- [ ] Comparaison de plusieurs modèles effectuée
- [ ] Analyse des erreurs réalisée

## Pour Aller Plus Loin

1. Testez d'autres features (titre extrait du nom, cabine, etc.)
2. Expérimentez avec le seuil de décision (au lieu de 0.5)
3. Utilisez la validation croisée (voir TP2)
4. Essayez d'autres algorithmes (SVM, Gradient Boosting)